# worked abit on a script here
copy from
https://www.kaggle.com/lopuhin/lyft-3d-join-all-lidars-annotations-from-scratch

i think some will get inspiration here

In [ ]:
%matplotlib inline
import json
import os.path

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from pyquaternion import Quaternion

In [ ]:
class Table:
    def __init__(self, data):
        self.data = data
        self.index = {x['token']: x for x in data}


DATA_ROOT = '/kaggle/input/3d-object-detection-for-autonomous-vehicles/'


def load_table(name, root=os.path.join(DATA_ROOT, 'train_data')):
    with open(os.path.join(root, name), 'rb') as f:
        return Table(json.load(f))

    
scene = load_table('scene.json')
sample = load_table('sample.json')
sample_data = load_table('sample_data.json')
ego_pose = load_table('ego_pose.json')
calibrated_sensor = load_table('calibrated_sensor.json')

In [ ]:
print(scene.data[0])
print(sample)
print(sample_data)

In [ ]:
my_scene = scene.data[0]
my_scene

And the first sample from that scene

In [ ]:
sample.index[my_scene['first_sample_token']],sample.index[sample.index[my_scene['first_sample_token']]['next']]

We'll use ``sample_data`` to fetch lidar images related to this sample. First check what is inside ``sample_data``

In [ ]:
sample_data.data[0],sample_data.data[1]

Now fetch lidar images related to the sample (note that to make it efficient you'll want to add an index like in official SDK)

In [ ]:
sample_data.data[0]
lidars = []
for x in sample_data.data:
    if x['sample_token'] == my_scene['first_sample_token'] and 'lidar' in x['filename']:
        lidars.append(x)
lidars

All lidars happen to have the same ego_pose (because they are on the same car?)

In [ ]:
{x['ego_pose_token'] for x in lidars}

Now let's load lidar's point data, we'll keep only first 3 columns (point coordinates)

In [ ]:
lidars_data = [
    # here, sorry
    np.fromfile(os.path.join(DATA_ROOT, x['filename']).replace('/lidar/', '/train_lidar/'), dtype=np.float32)
    .reshape(-1, 5)[:, :3] for x in lidars]
lidars_data[0].shape

# 3d triangulated surface plot

thats difficult to imagine something here.
i guess the more negative the higher...
so reversed that negativity

In [ ]:
lid=pd.DataFrame( lidars_data[0] )
lid['x']=(3*lid[0]).astype('int')
lid['y']=(3*lid[1]).astype('int')
lid['x']=(lid['x'])
lid['y']=(lid['y'])
lid=lid.groupby(['x','y']).mean()
lid=lid[lid[2]>-2]

lid=lid.reset_index()
fig = plt.figure()

import sys

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

import numpy
from numpy.random import randn
from scipy import array, newaxis

ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_trisurf(lid['x'].astype('int'),lid['y'].astype('int'),-(lid[2].astype('int')-0.6), cmap=cm.jet, linewidth=0)
fig.colorbar(surf)

ax.xaxis.set_major_locator(MaxNLocator(5))
ax.yaxis.set_major_locator(MaxNLocator(6))
ax.zaxis.set_major_locator(MaxNLocator(5))

fig.tight_layout()
ax.view_init(70, 40)
plt.draw()


# from sparse to bitmap

* adding some spectral color, we see now something interesting: the cars share the same color...
* i guess we see the buildings and we see trees
* the difficult thing is each object has a **radarshadow**

In [ ]:

from scipy.sparse import coo_matrix
coo = coo_matrix((lid[2]+2, ((lid['y']-lid['y'].min()).astype('int'),(lid['x']-lid['x'].min()).astype('int'))))
from matplotlib import pyplot as plt
plt.figure(figsize=(30,30))
plt.imshow(coo.todense(), interpolation='nearest',cmap=cm.jet)
plt.show()

# polar version

i don't know why i have to think cartesian, the image is probably constructed polar. Lets try to do something with that view
not its important imho to treat the data. Just to get the original view

In [ ]:
def cart2pol(x,y):

    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)*2
    return(rho, phi)
from scipy.sparse import coo_matrix

if True:
    simg=lidars_data[0]
    
    X=pd.DataFrame([])
    
    X[1]=lidars_data[0][:,0] #coo_matrix(simg).row
    X[0]=lidars_data[0][:,1] #coo_matrix(simg).col#.reshape(-1, 1)
    
    P=pd.DataFrame([])
    #P['r'],P['c']=cart2pol(X[0]-np.min(X[0]),X[1]-np.min(X[1]))
    P['r'],P['c']=cart2pol(X[0],X[1])
    P['d']=lidars_data[0][:,2]
    P['r']=(P['r']*3).astype('int').values
    P['c']=(P['c']*100).astype('int').values
    #Psp=P.to_sparse()
    print(P.shape)

the polar graphs shows some benefits
first a line is a circle around the car
the first line is the closest circle
cars are equally good visible, not as squares , but again as other colour


In [ ]:
#print(P.groupby(['r','c']).mean().reset_index().pivot(index='c',columns='r',values='d'))
plt.figure(figsize=(30,30))
plt.imshow(P.groupby(['r','c']).mean().reset_index().pivot(index='r',columns='c',values='d'), interpolation='nearest',cmap=cm.jet)
plt.show()


# visualize the distance lines touching the ground

peace of cake to see the cars hé

you can see the road isn't flat imho it goes down in one direction...

the middle of 0 and the ground 0.5 average, is somewhere 0.25

**seeing al the objects : are all the points below 0.25**

evidently i am not sure yet of all matrixes can be simplified like that
and i am not sure if a car is on a climbing/descending road, what visualisation i get here..; but probably it should not make very difference since a car drives on a surface...

In [ ]:
Pol=P.groupby(['r','c']).mean().reset_index().pivot(index='r',columns='c',values='d')
Pol=Pol-0.25
Pol=Pol[Pol>0]
plt.figure(figsize=(30,30))
plt.imshow(Pol, interpolation='nearest',cmap=cm.jet)
plt.show()

# and the objects are the other part of the matrix



In [ ]:
Pol=P.groupby(['r','c']).mean().reset_index().pivot(index='r',columns='c',values='d')
Pol=Pol-0.25
Pol=Pol[Pol<0]
plt.figure(figsize=(30,30))
plt.imshow(Pol, interpolation='nearest',cmap=cm.jet)
plt.show()

the bitmap is less clear

In [ ]:

coo = coo_matrix(( (lidars_data[0][:,2]-lidars_data[0][:,2].max()), ( (P['r']-P['r'].min()).astype('int'),(P['c']-P['c'].min()).astype('int'))))

from matplotlib import pyplot as plt
plt.figure(figsize=(30,30))
plt.imshow(coo.todense(), interpolation='nearest',cmap=cm.jet)
plt.show()

* svd is not so much informative at first sight
* you could use it to denoise, but lidar doesn't generate much noise i presume
* or enhance contrast
* but i don't get much out here

In [ ]:
from sklearn.decomposition import TruncatedSVD,SparsePCA
svd = TruncatedSVD(n_components=30, n_iter=20, random_state=42)
from sklearn.preprocessing import QuantileTransformer
Pol=P.groupby(['r','c']).mean().reset_index().pivot(index='r',columns='c',values='d')
Pol=Pol-0.25
Pol=Pol[Pol<0]
Pol=Pol[Pol>-1]
rng = np.random.RandomState(0)
qt = QuantileTransformer(n_quantiles=10, random_state=0)
PolQ=qt.fit_transform(Pol.fillna(+1)-1.5) 
print(Pol.shape)

u=svd.fit_transform(PolQ)
v=svd.components_
s=svd.singular_values_
plt.imshow( u  , interpolation='nearest',cmap=cm.jet)
plt.show()

plt.imshow( v , interpolation='nearest',cmap=cm.jet)
plt.show()
plt.figure(figsize=(30,30))

plt.imshow((u/s).dot(v) , interpolation='nearest',cmap=cm.jet)
plt.show()
plt.figure(figsize=(30,30))

plt.imshow(Pol-(u/s).dot(v) , interpolation='nearest',cmap=cm.jet)
plt.show()

plt.plot( s )
plt.show()

In [ ]:
[calibrated_sensor.index[x['calibrated_sensor_token']] for x in lidars]

Some helpers

In [ ]:
def rotate_points(points, rotation, inverse=False):
    assert points.shape[1] == 3
    q = Quaternion(rotation)
    if inverse:
        q = q.inverse
    return np.dot(q.rotation_matrix, points.T).T
    
def apply_pose(points, cs):
    """ Translate (lidar) points to vehicle coordinates, given a calibrated sensor.
    """
    points = rotate_points(points, cs['rotation'])
    points = points + np.array(cs['translation'])
    return points

def inverse_apply_pose(points, cs):
    """ Reverse of apply_pose (we'll need it later).
    """
    points = points - np.array(cs['translation'])
    points = rotate_points(points, np.array(cs['rotation']), inverse=True)
    return points

And let's vizualize all lidars. First let's see what happens if we don't use poses from the lidars

* you have to calibrate all lidars
* the colours reveal that each radar shows something, but the best is the topradar


In [ ]:
def viz_all_lidars(lidars, lidars_data, clip=50, skip_apply_pose=False):
    all_points = []
    all_colors = []
    Lid=pd.DataFrame([])
    for color, points, lidar in zip([[1, 0, 0, 0.5], [0, 1, 0, 0.5], [0, 0, 1, 0.5]], lidars_data, lidars):
        Temp=pd.DataFrame([])
        cs = calibrated_sensor.index[lidar['calibrated_sensor_token']]
        if not skip_apply_pose:
            points = apply_pose(points, cs)
        for xi in range(0,3):
            Temp[xi]=points[:,xi]*3
            Temp[xi]=Temp[xi].astype('int')
        all_points.append(points)
        all_colors.append(np.array([color] * len(points)))
        Lid=Lid.append(Temp)
    all_points = np.concatenate(all_points)
    all_colors = np.concatenate(all_colors)
    perm = np.random.permutation(len(all_points))
    all_points = all_points[perm]
    all_colors = all_colors[perm]
    #all_colors = np.round( all_points[perm,2]/3 )

    plt.figure(figsize=(20, 20))
    plt.axis('equal')
    plt.grid()
    plt.scatter(np.clip(all_points[:, 0], -clip, clip), np.clip(all_points[:, 1], -clip, clip), all_points[:, 2],c=all_colors)
    
    return Lid

Lid = viz_all_lidars(lidars, lidars_data, clip=40, skip_apply_pose=False)

In [ ]:

Lid2=Lid.groupby([0,1]).mean().reset_index()
Lid2=Lid2.pivot(index=0,columns=1,values=2)
print(Lid2.iloc[450].mean())

Lid2=Lid2-1.5   # floor sits at -1.5
Lid2=Lid2[Lid2>-1.5]
Lid2=Lid2[Lid2<12]  #if you want to see the trucks it has to be higher then 3
print(Lid2.iloc[450].mean())
plt.figure(figsize=(30,30))
plt.imshow(Lid2, interpolation='nearest',cmap=cm.jet)
plt.show()

# lets try some 3D...

found some sources here 
https://github.com/navoshta/KITTI-Dataset/blob/master/kitti-dataset.ipynb

and look what we have now... thats something that makes things more clear for me..
**you really see the shape of the cars.**

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

colors = {
    'Car': 'b',
    'Tram': 'r',
    'Cyclist': 'g',
    'Van': 'c',
    'Truck': 'm',
    'Pedestrian': 'y',
    'Sitter': 'k'
}
axes_limits = [
    [-20, 80], # X axis range
    [-20, 20], # Y axis range
    [-3, 10]   # Z axis range
]
axes_str = ['X', 'Y', 'Z']

point_size = 0.01 * (1. / 0.2)

def draw_box(pyplot_axis, vertices, axes=[0, 1, 2], color='black'):
    """
    Draws a bounding 3D box in a pyplot axis.
    
    Parameters
    ----------
    pyplot_axis : Pyplot axis to draw in.
    vertices    : Array 8 box vertices containing x, y, z coordinates.
    axes        : Axes to use. Defaults to `[0, 1, 2]`, e.g. x, y and z axes.
    color       : Drawing color. Defaults to `black`.
    """
    vertices = vertices[axes, :]
    connections = [
        [0, 1], [1, 2], [2, 3], [3, 0],  # Lower plane parallel to Z=0 plane
        [4, 5], [5, 6], [6, 7], [7, 4],  # Upper plane parallel to Z=0 plane
        [0, 4], [1, 5], [2, 6], [3, 7]  # Connections between upper and lower planes
    ]
    for connection in connections:
        pyplot_axis.plot(*vertices[:, connection], c=color, lw=0.5)
        
        
def draw_point_cloud(ax, title, axes=[0, 1, 2], xlim3d=None, ylim3d=None, zlim3d=None):
        """
        Convenient method for drawing various point cloud projections as a part of frame statistics.
        """

        ax.scatter(*np.transpose(velo_frame[:, axes]), s=point_size, c=velo_frame[:, 2], cmap='gray')
        ax.set_title(title)
        ax.set_xlabel('{} axis'.format(axes_str[axes[0]]))
        ax.set_ylabel('{} axis'.format(axes_str[axes[1]]))
        if len(axes) > 2:
            ax.set_xlim3d(*axes_limits[axes[0]])
            ax.set_ylim3d(*axes_limits[axes[1]])
            ax.set_zlim3d(*axes_limits[axes[2]])
            ax.set_zlabel('{} axis'.format(axes_str[axes[2]]))
        else:
            ax.set_xlim(*axes_limits[axes[0]])
            ax.set_ylim(*axes_limits[axes[1]])
        # User specified limits
        if xlim3d!=None:
            ax.set_xlim3d(xlim3d)
        if ylim3d!=None:
            ax.set_ylim3d(ylim3d)
        if zlim3d!=None:
            ax.set_zlim3d(zlim3d)
            
        #for t_rects, t_type in zip(tracklet_rects[frame], tracklet_types[frame]):
        #    draw_box(ax, t_rects, axes=axes, color=colors[t_type])

velo_frame=lidars_data[1]            
f2 = plt.figure(figsize=(15, 8))
ax2 = f2.add_subplot(111, projection='3d')    
draw_point_cloud(ax2, 'Lyft scan', xlim3d=(-30,30))
plt.show()

In [ ]:
velo_frame[:,2]

In [ ]:
def colorize(velo_fr2,schift):
        velo_fr2=velo_fr2+schift
        velo_col=[]
        velo_col=['g']
        for xi in range(1,len(velo_fr2)):
            if velo_fr2[xi]>0 and velo_fr2[xi]<3.25:  # ground level searched manually, actually the surface is not flat... i should create a surface function
                if velo_fr2[xi]>velo_fr2[xi-1]+0.001:  #increasing Z is red
                    velo_col.append('r')
                elif velo_fr2[xi]<velo_fr2[xi-1]-0.001: # decreasing Z is mauve
                    velo_col.append('m')
                else:
                    velo_col.append('b')
            elif velo_fr2[xi]>=3.25 :
                velo_col.append('g') #probably trees
            else:
                velo_col.append('y')

        return velo_col
    
def draw_point_cloud_color(ax, title, axes=[0, 1, 2], xlim3d=None, ylim3d=None, zlim3d=None):
        """
        Convenient method for drawing various point cloud projections as a part of frame statistics.
        """
            
        ax.scatter(*np.transpose(velo_frame[:, axes]), s=point_size, c=colorize(velo_frame[:,2],2), cmap='gray')
        ax.set_title(title)
        ax.set_xlabel('{} axis'.format(axes_str[axes[0]]))
        ax.set_ylabel('{} axis'.format(axes_str[axes[1]]))
        if len(axes) > 2:
            ax.set_xlim3d(*axes_limits[axes[0]])
            ax.set_ylim3d(*axes_limits[axes[1]])
            ax.set_zlim3d(*axes_limits[axes[2]])
            ax.set_zlabel('{} axis'.format(axes_str[axes[2]]))
        else:
            ax.set_xlim(*axes_limits[axes[0]])
            ax.set_ylim(*axes_limits[axes[1]])
        # User specified limits
        if xlim3d!=None:
            ax.set_xlim3d(xlim3d)
        if ylim3d!=None:
            ax.set_ylim3d(ylim3d)
        if zlim3d!=None:
            ax.set_zlim3d(zlim3d)
            
        #for t_rects, t_type in zip(tracklet_rects[frame], tracklet_types[frame]):
        #    draw_box(ax, t_rects, axes=axes, color=colors[t_type])

velo_frame=lidars_data[1]            
f2 = plt.figure(figsize=(15, 8))
ax2 = f2.add_subplot(111, projection='3d')    
draw_point_cloud_color(ax2, 'Lyft scan', xlim3d=(-30,30))
plt.show()

# polar version in 3d

In [ ]:
X=pd.DataFrame([])
    
X[1]=lidars_data[1][:,0] #coo_matrix(simg).row
X[0]=lidars_data[1][:,1] #coo_matrix(simg).col#.reshape(-1, 1)
    
P=pd.DataFrame([])
P['r'],P['c']=cart2pol(X[0],X[1])
P['d']=lidars_data[1][:,2]
point_size = 1

velo_frame=P.values           
f2 = plt.figure(figsize=(15, 8))
ax2 = f2.add_subplot(111, projection='3d')    
draw_point_cloud_color(ax2, 'Lyft polar 3d view ', xlim3d=(40,80))

plt.show()

def get_annotations(token):
    annotations = np.array(train_df.loc[token].PredictionString.split()).reshape(-1, 8)
    return {
        'point': annotations[:, :3].astype(np.float32),
        'wlh': annotations[:, 3:6].astype(np.float32),
        'rotation': annotations[:, 6].astype(np.float32),
        'cls': np.array(annotations[:, 7]),
    }

get_annotations(my_scene['first_sample_token']).keys()

In [ ]:
def viz_all_lidars_color(lidars, lidars_data, clip=50, skip_apply_pose=False):
    all_points = []
    all_colors = []
    Lid=pd.DataFrame([])

    for color, points, lidar in zip([[1, 0, 0, 0.5], [0, 1, 0, 0.5], [0, 0, 1, 0.5]], lidars_data, lidars):
        Temp=pd.DataFrame([])
        cs = calibrated_sensor.index[lidar['calibrated_sensor_token']]
        if not skip_apply_pose:
            points = apply_pose(points, cs)
        for xi in range(0,3):
            Temp[xi]=points[:,xi]*3
            Temp[xi]=Temp[xi].astype('int')
        all_points.append(points)
        #all_colors.append(np.array([color] * len(points)))
        Lid=Lid.append(Temp)
    all_points = np.concatenate(all_points)
    #all_colors = np.concatenate(all_colors)
    all_colors =colorize(all_points[:,2],-0.21)  #since data are calibrated i have to shift
    #print(len(all_points),len(all_colors))
    
    #perm = np.random.permutation(len(all_points))
    #all_points = all_points[perm]
    #all_colors = all_colors[perm]
    #all_colors = colorize( all_points[perm:,2]/3 ) 

    plt.figure(figsize=(20, 20))
    plt.axis('equal')
    plt.grid()
    plt.scatter(np.clip(all_points[:, 0], -clip, clip), np.clip(all_points[:, 1], -clip, clip), all_points[:, 2],c=all_colors)
    
    return Lid

Lid = viz_all_lidars_color(lidars, lidars_data, clip=40, skip_apply_pose=False)

In [ ]:
my_scene = scene.data[1]
print(my_scene,sample_data.data[1])
lidars = []
for x in sample_data.data:
    if x['sample_token'] == my_scene['first_sample_token'] and 'lidar' in x['filename']:
        lidars.append(x)
lidars

In [ ]:
ego_pose.index[lidars[0]['ego_pose_token']]

In [ ]:
def viz_annotation_centers(token, lidars, clip=50):
    # translate annotation points to the car frame
    ego_pose_token, = {x['ego_pose_token'] for x in lidars}
    ep = ego_pose.index[ego_pose_token]
    annotations = get_annotations(token)
    car_points = annotations['point'][annotations['cls'] == 'car']
    car_points = inverse_apply_pose(car_points, ep)
    
    plt.scatter(np.clip(car_points[:, 0], -clip, clip),
                np.clip(car_points[:, 1], -clip, clip),
                s=70,
                color='purple')
    
viz_all_lidars(lidars, lidars_data, clip=50)
viz_annotation_centers(my_scene['first_sample_token'], lidars, clip=50)

In [ ]:
viz_all_lidars(lidars, lidars_data, clip=20)
viz_annotation_centers(my_scene['first_sample_token'], lidars, clip=20)